1\. **Maximum wind speed prediction at the Sprogø station**

The exercise goal is to predict the maximum wind speed occurring every 50 years even if no measure exists for such a period. The available data are only measured over 21 years at the Sprogø meteorological station located in Denmark. 

The annual maxima are supposed to fit a normal probability density function. However such function is not going to be estimated because it gives a probability from a wind speed maxima. Finding the maximum wind speed occurring every 50 years requires the opposite approach, the result needs to be found from a defined probability. That is the quantile function role and the exercise goal will be to find it. In the current model, it is supposed that the maximum wind speed occurring every 50 years is defined as the upper 2% quantile.

By definition, the quantile function is the inverse of the cumulative distribution function. The latter describes the probability distribution of an annual maxima. In the exercise, the cumulative probability $p_i$ for a given year i is defined as $p_i = i/(N+1)$ with $N = 21$, the number of measured years. Thus it will be possible to calculate the cumulative probability of every measured wind speed maxima. From those experimental points, the scipy.interpolate module will be very useful for fitting the quantile function. Finally the 50 years maxima is going to be evaluated from the cumulative probability of the 2% quantile.

Practically, load the dataset:

```python
import numpy as np
max_speeds = np.load('max-speeds.npy')
years_nb = max_speeds.shape[0]
```

Compute then the cumulative probability $p_i$ (`cprob`) and sort the maximum speeds from the data. Use then the  UnivariateSpline from scipy.interpolate to define a quantile function and thus estimate the probabilities.

In the current model, the maximum wind speed occurring every 50 years is defined as the upper 2% quantile. As a result, the cumulative probability value will be:

```python
fifty_prob = 1. - 0.02
```

So the storm wind speed occurring every 50 years can be guessed as:

``` python
fifty_wind = quantile_func(fifty_prob)
```



In [74]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from scipy.interpolate import UnivariateSpline

max_speeds = np.load('max-speeds.npy')
years_nb = max_speeds.shape[0]
print(max(max_speeds), min(max_speeds))
print(max_speeds, "\n")
bins = np.linspace(20, 35, 25)
bin_centers = 0.5*(bins[1:] + bins[:-1])

mean, std = stats.norm.fit(max_speeds)

fifty_prob = 1 - 0.02

# Calculate cum distribution on discrete points of max_speeds, and then plot pcum probability (x) vs max_speeds (y). In correspondence of 0.98 we will find our predicted value

hist, edges = np.histogram(max_speeds, bins = 21, range = (22,33), density = True)
print(hist)
cumval = np.cumsum(hist)

print(hist.sum())
#print(len(cumval))
#print(cumval)

#plt.figure(figsize = (6,4))
#plt.scatter(cumval, np.sort(max_speeds))
#plt.show()

#pdf = stats.norm.pdf(bin_centers, loc = mean)
#cdf = stats.norm.cdf(bin_centers, loc = mean)
#ppf = stats.norm.ppf(fifty_prob, loc = mean)

#plt.figure(figsize=(6, 4))
#_,_,_ = plt.hist(max_speeds, bins, label="Annual maximum wind speeds", density=True)
#plt.plot(bin_centers, pdf, label="PDF")
#plt.plot(bin_centers, cdf, label="CDF")
#plt.plot(bin_centers, ppf, label="Quantile function")
#plt.legend()

#print(np.sort(max_speeds))
#years = np.arange(21)
#print(years)



32.97 22.46
[28.09 22.46 26.65 26.65 28.79 30.19 26.19 26.75 23.34 24.16 29.75 27.93
 26.68 23.85 32.97 25.79 26.37 25.28 26.73 23.2  26.76] 

[0.09090911 0.         0.18181821 0.09090911 0.09090911 0.
 0.09090911 0.18181821 0.36363643 0.27272732 0.         0.18181821
 0.09090911 0.         0.09090911 0.09090911 0.         0.
 0.         0.         0.09090878]
1.9090909090920574


2\. **Curve fitting of temperature in Alaska** 

The temperature extremes in Alaska for each month, starting in January, are given by (in degrees Celcius):

max:  17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18

min: -62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58

* Plot these temperature extremes.
* Define a function that can describe min and max temperatures. 
* Fit this function to the data with scipy.optimize.curve_fit().
* Plot the result. Is the fit reasonable? If not, why?
* Is the time offset for min and max temperatures the same within the fit accuracy?

3\. **2D minimization of a six-hump camelback function**

$$
f(x,y) = \left(4-2.1x^2+\frac{x^4}{3} \right) x^2 +xy + (4y^2 -4)y^2
$$

has multiple global and local minima. Find the global minima of this function.

Hints:

* Variables can be restricted to $-2 < x < 2$ and $-1 < y < 1$.
* Use numpy.meshgrid() and pylab.imshow() to find visually the regions.
* Use scipy.optimize.minimize(), optionally trying out several of its methods.

How many global minima are there, and what is the function value at those points? What happens for an initial guess of $(x, y) = (0, 0)$ ?


4\. **FFT of a simple dataset**

Performe a periodicity analysis on the lynxs-hares population

5\. **FFT of an image**

* Examine the provided image `moonlanding.png`, which is heavily contaminated with periodic noise. In this exercise, we aim to clean up the noise using the Fast Fourier Transform.
* Load the image using pylab.imread().
* Find and use the 2-D FFT function in scipy.fftpack, and plot the spectrum (Fourier transform of) the image. Do you have any trouble visualising the spectrum? If so, why?
* The spectrum consists of high and low frequency components. The noise is contained in the high-frequency part of the spectrum, so set some of those components to zero (use array slicing).
* Apply the inverse Fourier transform to see the resulting image.